In [73]:
import json
import spacy
from keybert import KeyBERT
import medspacy
from medspacy.preprocess import PreprocessingRule, Preprocessor
from medspacy.ner import TargetRule
from medspacy.context import ConTextRule
from medspacy.section_detection import Sectionizer
from medspacy.postprocess import PostprocessingRule, PostprocessingPattern, Postprocessor
from medspacy.postprocess import postprocessing_functions
from medspacy.visualization import visualize_ent, visualize_dep
from medspacy.ner import TargetMatcher

In [74]:
# Load medspacy model
nlp = medspacy.load()
target_matcher = nlp.get_pipe("medspacy_target_matcher")
target_rules = [
    # Symptoms
    TargetRule("cramping", "SYMPTOM"),
    TargetRule("mood swings", "SYMPTOM"),
    TargetRule("fatigue", "SYMPTOM"),
    TargetRule("headaches", "SYMPTOM"),
    TargetRule("breast tenderness", "SYMPTOM"),
    TargetRule("nausea", "SYMPTOM"),

    # Conditions
    TargetRule("menstrual cramps", "CONDITION"),
    TargetRule("endometriosis", "CONDITION"),
    TargetRule("polycystic ovary syndrome", "CONDITION", pattern=[
        {"LOWER": "pcos"}
    ]),
    TargetRule("uterine fibroids", "CONDITION"),
    TargetRule("irregular periods", "CONDITION"),
    TargetRule("menopause", "CONDITION"),
    TargetRule("premenstrual syndrome", "CONDITION", pattern=[
        {"LOWER": "pms"}
    ]),
    TargetRule("premenstrual dysphoric disorder", "CONDITION", pattern=[
        {"LOWER": "pmdd"}
    ]),
    TargetRule("dysmenorrhea", "CONDITION"),
    TargetRule("amenorrhea", "CONDITION"),
    TargetRule("oligomenorrhea", "CONDITION"),
    TargetRule("menorrhagia", "CONDITION"),
    TargetRule("metrorrhagia", "CONDITION"),
    TargetRule("dyspareunia", "CONDITION"),
    TargetRule("vaginismus", "CONDITION"),
    TargetRule("vulvodynia", "CONDITION"),
    TargetRule("vaginitis", "CONDITION"),
    TargetRule("cervicitis", "CONDITION"),
    TargetRule("pelvic inflammatory disease", "CONDITION", pattern=[
        {"LOWER": "pid"}
    ]), 
    TargetRule("bacterial vaginosis", "CONDITION", pattern=[
        {"LOWER": "bv"}
    ]),
    TargetRule("yeast infection", "CONDITION"),
    TargetRule("cervical dysplasia", "CONDITION"),
    TargetRule("cervical cancer", "CONDITION"),
    TargetRule("ovarian cysts", "CONDITION"),
    TargetRule("ovarian cancer", "CONDITION"),
    TargetRule("breast cancer", "CONDITION"),
    TargetRule("infertility", "CONDITION"),
    TargetRule("ectopic pregnancy", "CONDITION"),
    TargetRule("miscarriage", "CONDITION"),
    TargetRule("stillbirth", "CONDITION"),
    TargetRule("preterm birth", "CONDITION"),
    TargetRule("low birth weight", "CONDITION"),
    TargetRule("preeclampsia", "CONDITION"),
    TargetRule("gestational diabetes", "CONDITION"),
    TargetRule("postpartum depression", "CONDITION"),
    TargetRule("postpartum psychosis", "CONDITION"),
    TargetRule("postpartum thyroiditis", "CONDITION"),
    TargetRule("postpartum hemorrhage", "CONDITION"),
    TargetRule("postpartum preeclampsia", "CONDITION"),
    TargetRule("postpartum eclampsia", "CONDITION"),
    TargetRule("postpartum cardiomyopathy", "CONDITION"),
    TargetRule("postpartum infections", "CONDITION"),
    TargetRule("postpartum psychosis", "CONDITION"),
    TargetRule("postpartum anxiety", "CONDITION"),
    TargetRule("postpartum OCD", "CONDITION"),
    TargetRule("postpartum PTSD", "CONDITION"),
    TargetRule("postpartum bipolar disorder", "CONDITION"),
    TargetRule("postpartum schizophrenia", "CONDITION"),
    TargetRule("postpartum mania", "CONDITION"),
    TargetRule("postpartum hypomania", "CONDITION"),
    TargetRule("postpartum depression", "CONDITION"),
    TargetRule("postpartum anxiety", "CONDITION"),
    TargetRule("postpartum OCD", "CONDITION"),
    

    # Hormones
    TargetRule("estrogen", "HORMONE"),
    TargetRule("progesterone", "HORMONE"),
    TargetRule("follicle-stimulating hormone", "HORMONE", pattern=[
        {"LOWER": "fsh"}
    ]),
    TargetRule("luteinizing hormone", "HORMONE", pattern=[
        {"LOWER": "lh"}
    ]),

    # Treatments/Medications
    TargetRule("birth control", "TREATMENT"),
    TargetRule("hormone replacement therapy", "TREATMENT"),
    TargetRule("pain relievers", "TREATMENT"),
    TargetRule("antidepressants", "TREATMENT"),

    # General Concepts
    TargetRule("menstruation", "CONCEPT"),
    TargetRule("contraception", "CONCEPT"),
    TargetRule("pregnancy", "CONCEPT"),
    TargetRule("ovulation", "CONCEPT")
]
target_matcher.add(target_rules)
print(nlp.pipe_names)
print(nlp.get_pipe("medspacy_target_matcher").labels)

['medspacy_pyrush', 'medspacy_target_matcher', 'medspacy_context']
{'TREATMENT', 'SYMPTOM', 'HORMONE', 'CONCEPT', 'CONDITION'}


In [75]:
text = """ What is PMS, and how can you manage the symptoms? Here’s the lowdown on getting your life back on track.

Hands up if you regularly feel bloated, moody, hungry, tired, or a whole host of other irritating symptoms right before your period? Turns out, there could be a reason for that. Here we explore everything you need to know about premenstrual syndrome (PMS), from symptoms to treatment options.

Key takeaways
PMS is the umbrella term for the physical and emotional symptoms that can show up before your period.
Symptoms of PMS vary from person to person, but they can include bloating, food cravings, fatigue, and mood swings.
PMS symptoms usually start one to two weeks before your period and ease within four days of it starting.
Exercise, getting enough sleep, eating a balanced diet, and hormonal birth control can all help to manage symptoms of PMS.
Premenstrual syndrome: What is it? 
PMS is a group of different symptoms that occur a week or two before your period starts.

These symptoms can vary from person to person and can be physical, emotional, or a combination of both; more on these later.

PMS is often the butt of the joke, and it's become a bit of a stereotype that it makes women and people who menstruate “emotional.” It’s important to challenge this belief because it can trivialize PMS and make it hard for people who are struggling with PMS to feel heard.

If you have periods, it’s important to know the signs of PMS and how to deal with them. Keep reading to learn more.

What causes PMS?
It’s not entirely clear why some people experience PMS, but it’s likely due to the hormonal changes that happen during the menstrual cycle. These changes affect each person differently, so some people may experience more severe symptoms than others. Like many aspects of female health, PMS still isn’t completely understood, and we need more research to understand the exact cause.

How common is PMS?
PMS is pretty common. In fact, around 75% of women experience symptoms of PMS, so if you’re one of them, take some comfort in knowing that you’re definitely not alone.
"""
doc = nlp(text)

In [76]:
# Print entities recognized by SpaCy
for ent in doc.ents:
    print(ent.text, ent.label_)

PMS CONDITION
PMS CONDITION
PMS CONDITION
PMS CONDITION
fatigue SYMPTOM
mood swings SYMPTOM
PMS CONDITION
birth control TREATMENT
PMS CONDITION
PMS CONDITION
PMS CONDITION
PMS CONDITION
PMS CONDITION
PMS CONDITION
PMS CONDITION
PMS CONDITION
PMS CONDITION
PMS CONDITION
PMS CONDITION
PMS CONDITION


In [77]:
visualize_ent(doc)

In [78]:
# now for summarization
model = KeyBERT('distilbert-base-nli-mean-tokens')
keywords = model.extract_keywords(text)
print(keywords)

[('symptoms', 0.3415), ('menstrual', 0.3163), ('fatigue', 0.3023), ('tired', 0.3014), ('emotional', 0.2989)]


In [79]:
from transformers import pipeline
summarizer = pipeline("summarization",model="t5-base", tokenizer="t5-base")
summary = summarizer(text, max_length=200, min_length=30, do_sample=False)
print(summary[0]['summary_text'])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


PMS is a group of different symptoms that occur a week or two before your period . symptoms include bloating, food cravings, fatigue, and mood swings . around 75% of women experience symptoms of PMS .


In [82]:
# trying the summmary on one of the articles I scraped
# getting the text from contents

# Load the article from a JSON file
def load_article(file_path):
    with open(file_path, 'r') as file:
        articles = json.load(file)
    return articles

# Example path to your JSON file
article_file_path = '../article_results/articles.json'
articles = load_article(article_file_path)

# Assuming you want to use the first article
article = articles[0]
text = article['content']
print(text[:1000])

Every piece of content at Flo Health adheres to the highest editorial standards for language, style, and medical accuracy. To learn what we do to deliver the best health and lifestyle insights to you, check outour content review principles. Breast pain and nipple sensitivity are common symptoms at different points in your cycle. Here’s what can cause it and when to speak to your doctor. Changes in the way yourboobs and nipples lookand feel throughout the month can be confusing and uncomfortable. However, it’s not uncommon to experiencebreast pain or nipple sensitivityat different points in your cycle. You may be able to attribute this tohormone changes. Knowing what can be considered typical and when pain or sensitivity might be something to worry about can be difficult. So, here’s everything you need to know about breast pain andnipple sensitivity, particularly afterovulation. Key takeaways Is it normal for nipples to be sensitive after ovulation? Before we dig into whether it’s norma

In [83]:
summarizer = pipeline("summarization")

# Generate summary
summary = summarizer(text, max_length=150, min_length=30, length_penalty=2.0, truncation=True)

# Print the summary
print("Summary:")
print(summary[0]['summary_text'])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Summary:
 Breast pain and nipple sensitivity are common symptoms at different points in your cycle . It's not uncommon to experience breast pain or nipple sensitivity after ovulation . Not everyone who is pregnant will experience these symptoms, and not everyone will .


In [84]:
summarizer = pipeline("summarization",model="t5-base", tokenizer="t5-base")

# Generate summary
summary = summarizer(text, max_length=150, min_length=30, length_penalty=2.0, truncation=True)

# Print the summary
print("Summary:")
print(summary[0]['summary_text'])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Summary:
ovulation is the point in your menstrual cycle when one of your ovaries releases an egg . if your nipples feel sensitive to the touch, it can be a sign of pregnancy . it's not uncommon for a runner to experience breast pain and sensitivity at this point in their cycle .
